## Tensors
### Pytorch: Tensors

In [1]:
import torch

In [2]:
dtype = torch.float
device = torch.device("cpu")

In [3]:
# N은 배치 크기이며, D_in은 입력의 차원
# H는 은닉층의 차원이며, D_out은 출력 차원
N, D_in, H, D_out = 64, 1000, 100, 10

In [4]:
# 무작위의 입력과 출력 데이터를 생성
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

In [5]:
# 무작위로 가중치를 초기화
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

In [6]:
learning_rate = 1e-6
for t in range(500):
    # 순전파 단계: 예측값 y를 계산
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)
    
    # 손실(loss)을 계산하고 출력
    loss = (y_pred - y).pow(2).sum()
    if t % 50 == 49:
        print("epoch: ", t+1, "  loss:", loss.item())
    
    # 손실에 따른 w1, w2의 변화도를 계산하고 역전파
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)
    
    # 경사하강법(gradient descent)를 사용하여 가중치를 갱신
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

epoch:  50   loss: 11946.5234375
epoch:  100   loss: 423.5840148925781
epoch:  150   loss: 32.08344650268555
epoch:  200   loss: 3.257556915283203
epoch:  250   loss: 0.38011887669563293
epoch:  300   loss: 0.050181590020656586
epoch:  350   loss: 0.007351728156208992
epoch:  400   loss: 0.0013697340618818998
epoch:  450   loss: 0.0003710514574777335
epoch:  500   loss: 0.00014217176067177206


## Autograd
### PyTorch: Tensor와 autograd

`x`가 `x.requires_grad=True`인 Tensor라면 `x.grad`는 어떤 스칼라 값에 대한 `x`의 변화도를 갖는 또 다른 Tensor다.

In [7]:
import torch

dtype = torch.float
device = torch.device("cpu")

# N은 배치 크기이며, D_in은 입력의 차원
# H는 은닉층의 차원이며, D_out은 출력 차원
N, D_in, H, D_out = 64, 1000, 100, 10

In [8]:
# 무작위의 입력과 출력 데이터를 생성
# requires_grad = False로 설정하여 역전파 중에 이 Tensor들에 대한 변화도를
# 계산할 필요가 없음을 나타냄. (requires_grad의 defalut값은 False임)
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

In [9]:
# 무작위로 가중치를 초기화
# requires_grad=True로 설정하여 역전파 중에 이 Tensor들에 대한 변화도를
# 계산할 필요가 있음을 나타냄.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

In [10]:
learning_rate = 1e-6
for t in range(500):
    # 순전파 단계: Tensor 연산을 사용하여 예상되는 y값을 계산.
    # 이는 Tensor를 사용한 순전파 단계와 완전히 동일하지만, 역전파 단계를 별도로
    # 구현하지 않아도 되므로, 중간값들에 대한 정보를 가지고 있을 필요가 없어서
    # 정의하지 않음
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    
    # Tensor 연산을 사용하여 손실을 계산하고 출력
    #  loss는 (1,) 형태의 Tensor이며, loss.item()은 loss의 스칼라 값
    loss = (y_pred - y).pow(2).sum()
    if t % 50 == 49:
        print("epoch: ", t+1, "  loss:", loss.item())
    
    # autograd를 사용하여 역전파 단계를 계산.
    # 이는 requires_grad=True를 갖는 모든 Tensor에 대해 손실의 변화도를 계산
    # 이후 w1.grad와 w2.grad는 w1과 w2 각각에 대한 손실의 변화도를 갖는 Tensor가 됨
    loss.backward()
    
    # 경사하강법(gradient descent)을 사용하여 가중치를 수동으로 갱신
    # torch.no_grad()로 감싸는 이유는 가중치들이 requires_grad=True이지만
    # autograd에서는 이를 추적하지 않아야하기 때문
    # 다른 방법으로 weight.data 및 weight.grad.data를 조작하는 방법이있음
    # tensor.data가 tensor의 저장공간을 공유하기는 하지만, 이력을 추적하지는 않음
    # 또한, 이를 위해 torch.optim.SGD를 사용할 수 있음
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        # 가중치 갱신 후에는 수동으로 변화도를 0으로 만듦
        w1.grad.zero_()
        w2.grad.zero_()

epoch:  50   loss: 12525.3046875
epoch:  100   loss: 339.0439453125
epoch:  150   loss: 14.65766716003418
epoch:  200   loss: 0.7923321723937988
epoch:  250   loss: 0.049583449959754944
epoch:  300   loss: 0.0036387895233929157
epoch:  350   loss: 0.0004662868450395763
epoch:  400   loss: 0.00011778158659581095
epoch:  450   loss: 4.72370047646109e-05
epoch:  500   loss: 2.5624169211369008e-05


### Pytorch: 새 autograd 함수 정의하기

내부적으로, autograd의 기본(primitive) 연산자는 실제로 Tensor를 조작하는 2개의 함수다. forward 함수는 입력 Tensor로부터 출력 Tensor를 계산하고, backward 함수는 어떤 스칼라 값에 대한 출력 Tensor의 변화도를 전달받고, 동일한 스칼라 값에 대한 입력 Tensor의 변화도를 계산한다.

Pytorch에서 `torch.autograd.Function`의 서브클래스(subclass)를 정의하고 `forward`와 `backward` 함수를 구현함으로써 사용자 정의 autograd 연산자를 손쉽게 정의할 수 있다. 그 후, 인스턴스(instance)를 생성하고 이를 함수처럼 호출하여 입력 데이터를 갖는 Tensor를 전달하는 식으로 새로운 autograd 연산자를 사용할 수 있다.

이 예제에서는 ReLU로 비선형적(nonlinearity)으로 동작하는 사용자 정의 autograd 함수를 정의하고, 2-계층 신경망에 이를 적용해보자.

In [11]:
import torch

In [12]:
class MyReLU(torch.autograd.Function):
    """
    torch.autograd.Function을 상속받아 사용자 정의 autograd Function을 구현하고,
    Tensor 연산을 하는 순전파와 역전파 단계를 구현
    """
    
    @staticmethod
    def forward(ctx, input):
        """
        순전파 단계에서는 입력을 갖는 Tensor를 받아 출력을 갖는 Tensor를 반환
        ctx는 컨텍스트 객체(context object)로 역전파 연산을 위한 정보 저장에 사용
        ctx.save_for_backward method를 사용하여 역전파 단계에서 사용할 객체를 
        저장(cache)해 둘 수 있음
        """
        ctx.save_for_backward(input)
        return input.clamp(min=0)
    
    @staticmethod
    def backward(ctx, grad_output):
        """
        역전파 단계에서는 출력에 대한 손실의 변화도를 갖는 Tensor를 받고, 입력에
        대한 손실의 변화도를 계산
        """
        input  = ctx.saved_tensors[0]
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input

In [13]:
dtype = torch.float
device = torch.device("cpu")

# N은 배치 크기이며, D_in은 입력의 차원
# H는 은닉층의 차원이며, D_out은 출력 차원
N, D_in, H, D_out = 64, 1000, 100, 10

# 무작위의 입력과 출력 데이터를 생성
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 무작위로 가중치를 초기화
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

In [14]:
learning_rate = 1e-6

for t in range(500):
    # 사용자 정의 Function을 적용하기 위해 Function.apply 메소드를 사용
    # 여기에 "relu"라는 이름을 붙이자
    relu = MyReLU.apply
    # 꼭 여기 있어야할까? iteration 밖에 있으면 안될까?

    # 순전파 단계: Tensor 연산을 사용하여 예상되는 y 값을 계산
    # 사용자 정의 autograd 연산을 사용하여 ReLU를 계산
    y_pred = relu(x.mm(w1)).mm(w2)
    
    # 손실을 계산하고 출력
    loss = (y_pred - y).pow(2).sum()
    if t % 50 == 49:
        print("epoch: ", t+1, "  loss:", loss.item())
    
    # autograd를 사용하여 역전파 단계를 계산.
    loss.backward()
    
    # 경사하강법(gradient descent)을 사용하여 가중치를 수동으로 갱신
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        # 가중치 갱신 후에는 수동으로 변화도를 0으로 만듦
        w1.grad.zero_()
        w2.grad.zero_()

epoch:  50   loss: 14128.3369140625
epoch:  100   loss: 616.72216796875
epoch:  150   loss: 47.715450286865234
epoch:  200   loss: 4.376462936401367
epoch:  250   loss: 0.4345703125
epoch:  300   loss: 0.045466016978025436
epoch:  350   loss: 0.005184908397495747
epoch:  400   loss: 0.0008270065882243216
epoch:  450   loss: 0.00022427958901971579
epoch:  500   loss: 9.088731167139485e-05


In [15]:
learning_rate = 1e-6
# 사용자 정의 Function을 적용하기 위해 Function.apply 메소드를 사용
# 여기에 "relu"라는 이름을 붙이자
relu = MyReLU.apply
# 되는 듯?
for t in range(500):
    # 순전파 단계: Tensor 연산을 사용하여 예상되는 y 값을 계산
    # 사용자 정의 autograd 연산을 사용하여 ReLU를 계산
    y_pred = relu(x.mm(w1)).mm(w2)
    
    # 손실을 계산하고 출력
    loss = (y_pred - y).pow(2).sum()
    if t % 50 == 49:
        print("epoch: ", t+1, "  loss:", loss.item())
    
    # autograd를 사용하여 역전파 단계를 계산.
    loss.backward()
    
    # 경사하강법(gradient descent)을 사용하여 가중치를 수동으로 갱신
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        # 가중치 갱신 후에는 수동으로 변화도를 0으로 만듦
        w1.grad.zero_()
        w2.grad.zero_()

epoch:  50   loss: 4.7636036470066756e-05
epoch:  100   loss: 2.976922223751899e-05
epoch:  150   loss: 2.085721462208312e-05
epoch:  200   loss: 1.5468995115952566e-05
epoch:  250   loss: 1.1970103514613584e-05
epoch:  300   loss: 9.917773240886163e-06
epoch:  350   loss: 8.215315574489068e-06
epoch:  400   loss: 7.169872787926579e-06
epoch:  450   loss: 6.299682354438119e-06
epoch:  500   loss: 5.5774480642867275e-06


### TensorFlow: 정적 그래프(Static Graph)

PyTorch autograd는 Tensorflow와 많이 비슷해보입니다: 두 프레임워크 모두 연산 그래프를 정의하며, 자동 미분을 사용하여 변화도를 계산합니다. 두 프레임워크의 가장 큰 차이점은 Tensorflow의 연산 그래프는 정적 인데 반해, PyTorch는 동적 연산 그래프를 사용한다는 것입니다.

Tensorflow에서는 연산 그래프를 한 번 정의한 후 동일한 그래프를 계속해서 실행하며 가능한 다른 입력 데이터를 그래프에 전달합니다. PyTorch에서는 각각의 순전파 단계는 새로운 연산 그래프를 정의합니다.

정적 그래프는 그래프를 미리(upfront) 최적화할 수 있기 때문에 좋습니다; 예를 들어 프레임워크가 효율을 위해 일부 그래프 연산을 합치거나, 여러 GPU나 시스템(machine)에 그래프를 배포하는 전략을 제시하도록 결정할 수 있습니다. 만약 동일한 그래프를 계속 재사용하면, 초기의 값비싼 최적화 비용을 동일한 그래프를 반복하여 실행함으로써 상환(상쇄)할 수 있습니다.

정적 그래프와 동적 그래프는 제어 흐름(control flow) 측면에서도 다릅니다. 어떤 모델에서 각 데이터 지점(point)마다 다른 연산 연산을 수행하고 싶을 수 있습니다; 예를 들어 순환 신경망에서 각각의 데이터 지점마다 서로 다른 횟수만큼 펼칠(unroll) 수 있습니다; 이러한 펼침은 반복문(loop)으로 구현할 수 있습니다. 정적 그래프에서 반복문은 그래프의 일부가 돼야 합니다; 이러한 이유에서 Tensorflow는 그래프 내에 반복문을 포함하기 위해 tf.scan 과 같은 연산자를 제공합니다. 동적 그래프에서는 이러한 상황이 더 단순(Simple)해집니다: 각 예제에 대한 그래프를 즉석에서 작성하기 때문에, 일반적인 명령형(Imperative) 제어 흐름을 사용하여 각각의 입력에 따라 다른 계산을 수행할 수 있습니다.

## nn 모듈
### PyTorch: nn

연산 그래프와 autograd는 복잡한 연산자를 정의하고 도함수(derivative)를 자동으로 계산하는 매우 강력한 패러다임입니다; 하지만 규모가 큰 신경망에서는 autograd 그 자체만으로는 너무 낮은 수준(low-level)일 수 있습니다.

신경망을 구성할 때 종종 연산을 여러 계층 에 배열(arrange)하는 것으로 생각하는데, 이 중 일부는 학습 도중 최적화가 될 학습 가능한 매개변수 를 갖고 있습니다.

Tensorflow는 Keras, TensorFlow-Slim, 나 TFLearn 같은 패키지들이 연산 그래프를 더 높은 수준으로 추상화(higher-level abstraction)하여 제공하므로 신경망을 구축하는데 있어 유용합니다.

PyTorch에서는 nn 패키지가 동일한 목적으로 제공됩니다. nn 패키지는 신경망 계층(layer)들과 거의 동일한 Module 의 집합을 정의합니다. Module은 입력 Tensor를 받고 출력 Tensor를 계산하는 한편, 학습 가능한 매개변수를 갖는 Tensor 같은 내부 상태(internal state)를 갖습니다. nn 패키지는 또한 신경망을 학습시킬 때 주로 사용하는 유용한 손실 함수들도 정의하고 있습니다.


In [16]:
import torch

# N은 배치 크기이며, D_in은 입력의 차원
# H는 은닉층의 차원이며, D_out은 출력 차원
N, D_in, H, D_out = 64, 1000, 100, 10

# 무작위의 입력과 출력 데이터를 생성
x = torch.randn(N, D_in,)
y = torch.randn(N, D_out)

In [17]:
# nn 패키지를 사용하여 모델을 순차적 계층(sequence of layers)으로 정의
# nn.Sequential은 다른 Module들을 포함하는 Module로, 그 Module들을 순차적으로
# 적용하여 출력을 생성.
# 각각의 Linear Module은 선형 함수를 사용하여 입력으로부터 출력을 계산하고,
# 내부 Tensor에 가중치와 편향을 저장.
model = torch.nn.Sequential(
        torch.nn.Linear(D_in, H),
        torch.nn.ReLU(),
        torch.nn.Linear(H, D_out),
)

# 또한 nn 패키지에는 널리 사용되는 손실 함수들에 대한 정의도 포함되어있음
# 여기에서는 평균 제곱 오차(MSE)를 손실 함수로 사용
loss_fn = torch.nn.MSELoss(reduction='sum')

In [18]:
learning_rate = 1e-4
for t in range(500):
    # 순전파 단계: 모델에 x를 전달하여 예상되는 y 값을 계산
    # Module 객체는 __call__ 연산자를 덮어써(override) 함수처럼 호출할 수 있게 함
    # 이렇게 함으로써 입력 데이터의 Tensor를 Module에 전달하여 출력 데이터의
    # Tensor를 생성
    y_pred = model(x)
    
    # 손실은 계산하고 출력한다. 예측한 y와 정답인 y를 갖는 Tensor들을 전달하고,
    # 손실 함수는 손실 값을 갖는 Tensor를 반환한다.
    loss = loss_fn(y_pred, y)
    if t % 50 == 49:
        print("epoch: ", t+1, "  loss:", loss.item())
        
    # 역전파 단계를 실행하기 전에 변화도를 0으로 만든다.
    model.zero_grad()
    
    # 역전파 단계: 모델의 학습 가능한 모든 매개변수에 대해 손실의 변화도를
    # 계산. 내부적으로 각 Module의 매개변수는 requires_grad=True 일 때 Tensor 내에 저장되므로,
    # 이 호출은 모든 모델의 모든 학습 가능한 매개변수의 변화도를 계산한다.
    loss.backward()
    
    # 경사하강법(gradient descent)를 사용하여 가중치를 갱신. 각 매개변수는 Tensor이므로
    # 이전에 했던 것과 같이 변화도에 접근할 수 있음
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

epoch:  50   loss: 41.30939483642578
epoch:  100   loss: 3.3156707286834717
epoch:  150   loss: 0.44761595129966736
epoch:  200   loss: 0.07103249430656433
epoch:  250   loss: 0.012642757035791874
epoch:  300   loss: 0.0025322295259684324
epoch:  350   loss: 0.0005619027651846409
epoch:  400   loss: 0.0001362628972856328
epoch:  450   loss: 3.556507363100536e-05
epoch:  500   loss: 9.822014362725895e-06


### PyTorch: optim

지금까지는 (autograd의 추적 기록을 피하기 위해 torch.no_grad () 또는 .data 를 사용하는 식으로) 학습 가능한 매개변수를 갖는 Tensor를 직접 조작하며 모델의 가중치를 갱신하였습니다. 이것은 확률적 경사 하강법(SGD)과 같은 간단한 최적화 알고리즘에서는 크게 부담이 되지는 않지만, 실제로 신경망을 학습할 때는 주로 AdaGrad, RMSProp, Adam 등과 같은 좀 더 정교한 Optimizer를 사용하곤 합니다.

PyTorch의 optim 패키지는 최적화 알고리즘에 대한 아이디어를 추상화하고 일반적으로 사용하는 최적화 알고리즘의 구현체(implementation)를 제공합니다.

이 에제에서는 지금까지와 같이 nn 패키지를 사용하여 모델을 정의하지만, optim 패키지가 제공하는 Adam 알고리즘을 이용하여 모델을 최적화하겠습니다:

In [19]:
import torch

# N은 배치 크기이며, D_in은 입력의 차원
# H는 은닉층의 차원이며, D_out은 출력 차원
N, D_in, H, D_out = 64, 1000, 100, 10

# 무작위의 입력과 출력 데이터를 생성
x = torch.randn(N, D_in,)
y = torch.randn(N, D_out)

# nn 패키지를 사용하여 모델과 손실 함수를 정의
model = torch.nn.Sequential(
        torch.nn.Linear(D_in, H),
        torch.nn.ReLU(),
        torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# optim 패키지를 사용하여 모델의 가중치를 갱신할 Optimizer를 정의
# 여기서는 Adam을 사용. optim 패키지는 다른 다양한 최적화 알고리즘을 포함하고 있음
# Adam 생성자의 첫번째 인자는 어떤 Tensor가 갱신되어야 하는지를 받음
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(500):
    # 순전파 단계: 모델에 x를 전달하여 예상되는 y 값을 계산
    y_pred = model(x)
    
    # 손실을 계산하고 출력
    loss = loss_fn(y_pred, y)
    if t % 50 == 49:
        print("epoch: ", t+1, "  loss:", loss.item())
        
    # 역전파 단계 전에, optimizer 객체를 사용하여 (모델의 학습 가능한 가중치인)
    # 갱신할 변수들에 대한 모든 변화도를 0으로 만듦. 이렇게 하는 이유는
    # 기본적으로 .backward()를 호출할 때마다 변화도가 버퍼(buffer)에 (덮어씌워지지 않고)
    # 누적되기 때문. 더 자세한 내용은 torch.autograd.backward에 대한 문서를 보면
    # 알 수 있음
    optimizer.zero_grad()

    # 역전파 단계: 모델의 매개변수에 대한 손실의 변화도를 계산한다.
    loss.backward()
    
    # optimizer의 step 메소드를 호출하면 매개변수가 갱신
    optimizer.step()

epoch:  50   loss: 183.71543884277344
epoch:  100   loss: 42.39488220214844
epoch:  150   loss: 5.6100993156433105
epoch:  200   loss: 0.4225078821182251
epoch:  250   loss: 0.021901104599237442
epoch:  300   loss: 0.0010471625719219446
epoch:  350   loss: 4.8450285248691216e-05
epoch:  400   loss: 1.619708427824662e-06
epoch:  450   loss: 3.40509345164719e-08
epoch:  500   loss: 6.094150739599513e-10


### PyTorch: 사용자 정의 nn.Module

때때로 기존 모듈의 구성(sequence)보다 더 복잡한 모델을 구성해야 할 때가 있다. 이럴 때는 `nn.Module`의 서부 클래스로 새 모듈을 정의하고, 입력 Tensor를 받아 다른 모듈 또는 Tensor의 autograd 연산을 사용하여 출력 Tensor를 만드는 `forward`를 정의한다.

In [20]:
import torch    

In [21]:
class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        생성자에서 2개의 nn.Linear 모듈을 생성하고, 멤버 변수로 지정
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)
    
    def forward(self, x):
        """
        순전파 함수에서는 입력 데이터의 Tensor를 받고 출력 데이터의 Tensor를 반환
        Tensor 상의 임의의 연산자뿐만 아니라 생성자에서 정의한 Module도 사용할 수
        있음
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred

In [22]:
# N은 배치 크기이며, D_in은 입력의 차원
# H는 은닉층의 차원이며, D_out은 출력 차원
N, D_in, H, D_out = 64, 1000, 100, 10

# 무작위의 입력과 출력 데이터를 생성
x = torch.randn(N, D_in,)
y = torch.randn(N, D_out)

In [23]:
# 앞에서 정의한 클래스를 생성하여 모델을 구성
model = TwoLayerNet(D_in, H, D_out)

# 손실 함수와 Optimizer를 만듦. SGD 생성자에 model.parameters()를 호출하면
# 모델의 멤버인 2개의 nn.Linear 모듈의 학습 가능한 매개변수들이 포함된다.
criterion = torch.nn.MSELoss(reduction="sum")
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
for t in range(500):
    # 순전파 단계: 모델에 x를 전달하여 예상되는 y 값을 계산
    y_pred = model(x)
    
    # 손실을 계산하고 출력
    loss = criterion(y_pred, y)
    if t % 50 == 49:
        print("epoch: ", t+1, "  loss:", loss.item())
    
    # 변화도를 0으로 만들고, 연전파 단계를 수행하고, 가중치를 갱신
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

epoch:  50   loss: 36.87100601196289
epoch:  100   loss: 2.2368500232696533
epoch:  150   loss: 0.2537204623222351
epoch:  200   loss: 0.03923201188445091
epoch:  250   loss: 0.006858561187982559
epoch:  300   loss: 0.0012878882698714733
epoch:  350   loss: 0.00025374407414346933
epoch:  400   loss: 5.193155084270984e-05
epoch:  450   loss: 1.0984310392814223e-05
epoch:  500   loss: 2.3942873212945415e-06


### PyTorch: 제어 흐름(Control Flow) + 가중치 공유(Weight Sharing)

동적 그래프와 가중치 공유의 예로, 매우 이상한 모델을 구현해보자. 각 순전파 단계에서 많은 은닉 계층을 갖는 완전히 연결(fully-connected)된 ReLU 신경망이 무작위로 0-3 사이의 숫자를 선택하고, 가장 안쪽(innermost)의 은닉층들을 계산하기 위해 동일한 가중치를 여러 번 재사용한다.

이 모델에서는 일반적인 Python 제어 흐름을 사용하여 반복(loop)을 구현할 수 있으며, 순전파 단계를 정의할 때, 단지 동일한 Module을 여러번 재사용함으로써 내부(innermost) 계층들 간의 가중치 공유를 구현할 수 있다.

In [24]:
import random
import torch


class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        생성자에서 순전파 단계에서 사용할 3개의 nn.Linear 인스턴스를 생성
        """
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)
    
    def forward(self, x):
        """
        모델의 순전파 단계에서, 무작위로 0, 1, 2 또는 3 중에 하나를 선택하고
        은닉층을 계산하기 위해 여러번 사용한 middle_linear Module을 재사용한다.
        
        각 순전파 단계는 동적 연산 그래프를 구성하기 때문에, 모델의 순전파 단계를
        정의할 때 반복문이나 조건문과 같은 일반적인 python 제어 흐름 연산자를 사용할
        수 있다.
        
        여기서 연산 그래프를 정의할 때 동일 Module을 여러번 재사용하는 것이 가능하다는 것을
        알 수 있는데, 이것이 각 Module을 한 번씩만 사용할 수 있었던 Lua Torch보다 크게 개선된
        부분이다.
        """
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred

In [25]:
# N은 배치 크기이며, D_in은 입력의 차원
# H는 은닉층의 차원이며, D_out은 출력 차원
N, D_in, H, D_out = 64, 1000, 100, 10

# 무작위의 입력과 출력 데이터를 생성
x = torch.randn(N, D_in,)
y = torch.randn(N, D_out)

# 앞에서 정의한 클래스를 생성하여 모델을 구성
model = TwoLayerNet(D_in, H, D_out)

# 손실 함수와 Optimizer를 만듦. 
# 여기서는 momentum optimizer를 사용해보자
criterion = torch.nn.MSELoss(reduction="sum")
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)

for t in range(500):
    # 순전파 단계: 모델에 x를 전달하여 예상되는 y 값을 계산
    y_pred = model(x)
    
    # 손실을 계산하고 출력
    loss = criterion(y_pred, y)
    if t % 50 == 49:
        print("epoch: ", t+1, "  loss:", loss.item())
    
    # 변화도를 0으로 만들고, 연전파 단계를 수행하고, 가중치를 갱신
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()    

epoch:  50   loss: 1.581527590751648
epoch:  100   loss: 0.006852217484265566
epoch:  150   loss: 3.466135240159929e-05
epoch:  200   loss: 2.1005631367643218e-07
epoch:  250   loss: 1.0868349553661005e-09
epoch:  300   loss: 8.567129644587723e-12
epoch:  350   loss: 2.7710912991335546e-12
epoch:  400   loss: 2.4731768282632016e-12
epoch:  450   loss: 2.2664498316310455e-12
epoch:  500   loss: 2.2016162764398883e-12
